In [6]:
import numpy as np
import sys
from collections import defaultdict

if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.blackjack import BlackjackEnv

In [7]:
env = BlackjackEnv()

In [8]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Crea una política epsilon-greedy basado en una q-función (función de valor estado-acción) y un epsilon dados.
    
    Argumentos:
        Q: un diccionario que mapea cada estado/observación s a un array de numpy Q[s] = array([v_0, v_1, ... , v_nA]) de longitud nA
        que para un índice a del array contiene el valor v_a de tomar la acción a en el estado s. 
        (en nuestra notación de la clase q(s,a))
         
        epsilon: probabilidad de seleccionar una acción aleatoria (obliga a explorar), valor entre 0 y 1.
        
        nA: número de acciones en el entorno
    
    Retorna:
        Una función que dada una observación como argumento, retorna una política (un array de numpy de longitud nA)
        con probabilidades para cada acción. La política es tal que toma la mejor acción según Q con probabilidad (1-epsilon)
        y toma una acción al azar con probabilidad epsilon
        
    """
    def policy_fn(observation):
        policy = np.full(nA, epsilon / nA)
        best_action = Q[observation].argmax()
        policy[best_action] += 1 - epsilon
        return policy
        
    return policy_fn

In [9]:
def mc_control_epsilon_greedy(env, num_episodes, discount_factor=1.0, epsilon=0.1):
    """
    Control Monte Carlo utiliando políticas epsilon-greedy
    Encuentra la política epsilon-greedy óptima.
    
    Argumentos:
        env: ambiente de OpenAI gym.
        num_episodes: número de episodios a samplear.
        discount_factor: factor de descuento gama.
        epsilon: probabilidad de samplear una acción aleatoria. Valor entre 0 y 1.
    
    Retorna:
        Una tupla (Q, policy)
        Q es un diccionario que mapea cada estado/observación s a un array de numpy Q[s] = array([v_0, v_1, ... , v_nA]) de longitud nA
        que para un índice a del array contiene el valor v_a de tomar la acción a en el estado s. 
        (en nuestra notación de la clase q(s,a))
        policy es una función que toma una observación/estado como argumento y retorna un array numpy 
        de longitud nA con las probabilidades de cada acción
        
    """
    
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
        
    for ep in range(num_episodes):
        if ep % 1000 == 0:
            print(ep)
        
        state = env.reset()        
        steps = []
        
        for t in range(100):
            prob_actions = policy(state)
            action = np.random.choice(prob_actions.shape[0], p=prob_actions)
            next_state, reward, done, info = env.step(action)
            steps.append((state, action, reward))
            state = next_state
            if done:
                break
                
        for q in range(t, 0, -1):
            state, action, reward = steps[q]
            
            G = 0
            if (state, action, reward) not in steps[0:q]:
                G = reward + discount_factor * G
                
                if (state, action) in returns_sum:
                    returns_sum[(state, action)] += G
                else:
                    returns_sum[(state, action)] = G
                    
                if (state, action) in returns_count: 
                    returns_count[(state, action)] += 1
                else:
                    returns_count[(state, action)] = 1
                    
                Q[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]
    
    return Q, policy

In [10]:
Q, policy = mc_control_epsilon_greedy(env, num_episodes=500000, epsilon=0.1)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [17]:
def strategy(observation):
    prob = policy(observation)
    return np.random.choice([0, 1], p=prob)

In [18]:
def print_observation(observation):
    score, dealer_score, usable_ace = observation
    print("Player Score: {} (Usable Ace: {}), Dealer Score: {}".format(
          score, usable_ace, dealer_score))

In [19]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        print_observation(observation)
        action = strategy(observation)
        print("Taking action: {}".format( ["Stick", "Hit"][action]))
        observation, reward, done, _ = env.step(action)
        if done:
            print_observation(observation)
            print("Game end. Reward: {}\n".format(float(reward)))
            break

Player Score: 17 (Usable Ace: True), Dealer Score: 9
Taking action: Hit
Player Score: 20 (Usable Ace: True), Dealer Score: 9
Taking action: Stick
Player Score: 20 (Usable Ace: True), Dealer Score: 9
Game end. Reward: 1.0

Player Score: 13 (Usable Ace: False), Dealer Score: 10
Taking action: Hit
Player Score: 14 (Usable Ace: False), Dealer Score: 10
Taking action: Hit
Player Score: 21 (Usable Ace: False), Dealer Score: 10
Taking action: Stick
Player Score: 21 (Usable Ace: False), Dealer Score: 10
Game end. Reward: 1.0

Player Score: 18 (Usable Ace: False), Dealer Score: 3
Taking action: Stick
Player Score: 18 (Usable Ace: False), Dealer Score: 3
Game end. Reward: 0.0

Player Score: 16 (Usable Ace: False), Dealer Score: 3
Taking action: Stick
Player Score: 16 (Usable Ace: False), Dealer Score: 3
Game end. Reward: -1.0

Player Score: 21 (Usable Ace: True), Dealer Score: 8
Taking action: Stick
Player Score: 21 (Usable Ace: True), Dealer Score: 8
Game end. Reward: 1.0

Player Score: 21 (Usa